In [1]:
import pandas as pd
import openai
import time

pd.options.display.max_columns = None

client = openai.Client()

data_directory = 'data_sample/acre_beef_supply_chain_data.csv'
df = pd.read_csv(data_directory, index_col=0)
display(df.sample(2))

file = client.files.create(file=open(data_directory, "rb"), purpose="assistants")
print(file.id)

assistant = client.beta.assistants.create(
  name="Analista de Dados",
  instructions="Você é um analista que analisa dados sobre a cadeia de suprimentos de carne bovina brasileira",
  tools=[{"type": "code_interpreter"}],
  tool_resources={"code_interpreter": {"file_ids": [file.id]}},
  model="gpt-3.5-turbo-0125",
)

,year,biome,country_of_production,product_type,forest_500_beef,zero_deforestation_brazil_beef,state_of_production,municipality_of_production,logistics_hub,exporter,exporter_group,importer,importer_group,country_of_destination,economic_bloc,volume,fob,cattle_deforestation_5_year_total_exposure,co2_gross_emissions_cattle_deforestation_5_year_total_exposure,co2_net_emissions_cattle_deforestation_5_year_total_exposure,land_use,state_of_production_trase_id,municipality_of_production_trase_id,country_of_production_trase_id,country_of_destination_trase_id
4561819,2019,AMAZONIA,BRAZIL,BEEF OFFALS,NaN,NONE,ACRE,SENADOR GUIOMARD,UNKNOWN,PLENA ALIMENTOS,PLENA ALIMENTOS,EVER ALLIANCE INTERNATIONAL LTD,EVER ALLIANCE INTERNATIONAL LTD,CHINA (HONG KONG),CHINA (HONG KONG),0.355048,953.835925,0.083436,50.047918,46.986126,5.146055,BR-12,BR-1200450,BR,HK
4031835,2017,AMAZONIA,BRAZIL,BEEF BONELESS,NaN,NONE,ACRE,PLACIDO DE CASTRO,UNKNOWN,MICHI FOODS PRODUCAO COMERCIO DISTRIBUICAO IMP...,MICHI FOODS PRODUCAO COMERCIO DISTRIBUICAO IMP...,HATTON INTERNATIONAL,HATTON INTERNATIONAL,CHINA (HONG KONG),CHINA (HONG KONG),0.937011,2119.369327,0.376304,223.983615,210.174231,19.151909,BR-12,BR-1200385,BR,HK


file-QnzjJASRRDa3Fy5GQyPSZ5


In [10]:
question = "Gere um gráfico de rosca, bonito e estilizado, com a relação do volume exportado por ano (`volume` x `year`)"

# Criação da thread
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=question
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Nome do usuário Premium"
)

# Aguarda thread rodar
while run.status in ['queued', "in_progress", "cancelling"]:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

# Verifica a resposta
if run.status == "completed":
  msgs = client.beta.threads.messages.list(thread_id=thread.id)
  print(msgs)
else: print(f"Error {run.status}")

# Análise dos passos do modelo
run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)


# sobreescreve o conteúdo pré existente no arquivo
with open("data_sample/csv_reader_steps_assistant.md", "w") as f: f.write("")

for step in run_steps.data[::-1]:
  
  # abre o arquivo novamente no formato concatenação (append)
  with open("data_sample/csv_reader_steps_assistant.md", "a") as f: 
    print("\n### Step", step.step_details.type)
    f.write(f"### Step {step.step_details.type}\n")
    
    if step.step_details.type == "tool_calls":
      for tool_call in step.step_details.tool_calls:
        print("\n```", tool_call.code_interpreter.input, "```\n")
        f.write(f"\n```\n{tool_call.code_interpreter.input}\n```\n")
        
    elif step.step_details.type == "message_creation":
      message = client.beta.threads.messages.retrieve(
        thread_id=thread.id,
        message_id=step.step_details.message_creation.message_id
      )
      if message.content[0].type == "text": 
          print(message.content[0].text.value)
          f.write(f"\n{message.content[0].text.value}\n")
          
      elif message.content[0].type == "image_file": 
        file_id = message.content[0].image_file.file_id
        img_data = client.files.content(file_id)
        file_id = "sample_image"
        
        with open(f"data_sample/{file_id}.png", "wb") as fx:
          fx.write(img_data.read())
          f.write(f"\nImagem {file_id} salva!\n ![]({file_id}.png)")
          
          print(f"Imagem {file_id} salva!")

SyncCursorPage[Message](data=[Message(id='msg_dOBm76rt0xrVsUXhuqMFsyUj', assistant_id='asst_GCMCFTuInGkc3g93ID8cC0QZ', attachments=[], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-4HTjfBtQsaugdCAxMcDpgU', detail=None), type='image_file'), TextContentBlock(text=Text(annotations=[], value='Aqui está o gráfico de rosca que mostra a relação do volume exportado por ano. Cada fatia representa a proporção do volume exportado em um determinado ano.\n\nSe precisar de mais alguma coisa, estou à disposição para ajudar!'), type='text')], created_at=1742911755, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_UAEYT2Ln9OdgINMfixmlMPRn', status=None, thread_id='thread_omsL7NM4WCEyLDJbRnJB4vxB'), Message(id='msg_zwedpaBePYARKFY2iDXzIbz9', assistant_id='asst_GCMCFTuInGkc3g93ID8cC0QZ', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Parece que ocorreu um e